# Retrieval-Augmented Generation (RAG)

* [Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks](https://arxiv.org/abs/2005.11401)
* FLARE: [Active Retrieval Augmented Generation](https://arxiv.org/abs/2305.06983)
* [Архитектура RAG: полный гайд](https://habr.com/ru/amp/publications/791034/)
* [RAG (Retrieval Augmented Generation) — простое и понятное объяснение](https://habr.com/ru/articles/779526/)
* [RAG From Scratch](https://github.com/langchain-ai/rag-from-scratch?tab=readme-ov-file)

## Компоненты RAG-системы

RAG - это техника повышения точности и надежности LLM с использованием документов, полученных из внешних источников. 

Процесс генерации информации при помощи RAG выглядит следующим образом:
1. Пользователь составляет **запрос**
2. Запрос отправляется в **поисковик**
    * Определяется процесс поиска и извлечения документов по запросу:
        * **Retrievals**
        * **Векторная база данных**
3. Извлеченные документы отправляются в **генератор**
    * Комбинация входного запроса и извлеченных документов
4. Итоговый ответ

## Векторная база данных

Векторные базы данных хранят векторы контекста (эмбеддинги). На основе такой базы знаний производится **семантический поиск**, который помогает определить, насколько документы релевантны запросу.

1. Правильно нарезаем базу знаний на чанки для базы данных
    * По символам? По абзацам? Логически? С перекрытием?
    * Важные параметры: количество чанков, размер чанка
2. Какой эмбедер выбрать
    * Нужно ли его дообучать?
    
Примеры векторных баз:
* FAISS
* ChromaDB
* QDrant
* ElasticSearch
* Weaviate
* Milvus
* Pinecone

## Retrievals

[Ретриверы из langchain_community](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/retrievers)

* **Dense Retriever (embdedding similarity)** - используют трансформеры, например, BERT. Сходство между закодированными векторами вычисляется при помощи близости (cosine similarity).
    * Стандартный ретривер - косинусная близость вектора запроса и вектора документа
* **Sparse Retriever (разряженные вектора)** - традиционные методы информационного поиска, основанные на частотности
    * TF‑IDF ретривер
    * BM25 ретривер

Можно использовать какой-то определенный ретривер, а можно построить **ансамбль ретриверов** при помощи EnsembleRetriever:
* Результаты работы всех ретриверов объединяются
* Общий пул документов ранжируется, например, при помощи, Reciprocal Rank Fusion

> **MultiQueryRetriever**: перефразируем запрос несколько раз, извлечем документы по всем перефразированным вопросам, затем отранжируем общий пул документов

## Rank Fusion Techniques

Вообще, очень важно в каком порядке LLM увидит результаты поиска. Чем выше наиболее релевантный документ, тем лучше будет финальная генерация. В случае использования одного ретривера выборка ранжируется по метрике этого ретривера. Если же ретриверов несколько, то нужно использовать **реранкер**.

Внутри EnsembleRetriever используется Reciprocal Rank Fusion. Основная его идея - придать большее значение элементам, занимающим более высокие позиции в каждом индивидуальном наборе результатов поиска. Формула RRF: 1 / (k + rank), где rank - позиция элемента в конкретном поиске, а k - константа (по умолчанию k=60). Оценки для каждого элемента в разных наборах затем суммируются для получения итоговой оценки. Так мы избавляемся от привязки к метрике конкретного ретривера и подчеркиваем значимость наиболее высоко отранжированных элементов.

Другие реранкеры:
* Cohere Rerank

## RAG-Fusion: Multi-Query Retrieval + Reranker (Rank Fusion Techniques)

[RAG-Fusion: Multi-Query Retrieval & Rank Fusion Techniques](https://docsbot.ai/article/advanced-rag-techniques-multiquery-and-rank-fusion)

Что будет, если вместо простой функции ранжирования документов использовать ML-модель? Идея похожа на описанную в предыдущем пункте:
* Будем формулировать несколько вариантов запроса пользователя
* Искать по каждому запросу документы в базе данных
* Ранжировать и объединять результаты при помощи **Cross-Encoder**

Вообще, векторные базы данных используют Bi-encoder-ы, чтобы вычислить похожесть двух векторов. На вход Bi-encoder принимает 1 документ и преобразует его в вектор. Тогда схема работы RAG-системы с Bi-encoder выглядит так:
```
Текст А --> BERT преобразование в вектор --> косинусная мера близости <-- BERT преобразование в вектор <-- Текст Б
```

Cross-Encoder принимает на вход 2 документа и возвращает их релевантность (similarity) относительно друг друга. Схема работы Cross-Encoder выглядит иначе:
```
Текст А, Текст Б --> BERT --> Классификатор --> Релевантность 0..1
```

Точность работы Cross-Encoder, как правило, выше, чем у Bi-encoder. Идея заключается в том, чтобы извлечь из базы побольше результатов, а затем отранжировать их при помощи Cross-Encoder и вернуть на вход LLM 3-5 из них.

> **Интент-классификатор**: можно классифицировать документы на вопросы, жалобы, просьбы и т.д.

## Дообучение LLM

В задаче RAG на вход LLM поступают пары сущностей: входной запрос и контекст, состоящий из документов. LLM можно натюнить на такой вход.

## Retrieval Augmented Language Model based Prediction

Будем на шаге поиска вместо обычно передачи результатов на генерацию, генерировать примеры ответов для LLM (few-shot prompting), заставляя ее динамически обучаться отвечать на поставленный вопрос.

## Промпт-инжениринг

#### Корректный системный промпт

Источник: [GigaChain работа с промптами](https://github.com/ai-forever/gigachain/tree/master/hub/prompts)

```
input_variables: []
output_parser: null
template: 'Ты - система информационного поиска. Тебе дан вопрос и релевантные отрывки текста из нескольких документов.
Создай краткий и информативный ответ (не более 150 слов) на заданный вопрос,
основываясь исключительно на приведенных отрывках документов. Ты должна использовать только информацию из приведенных отрывков.
Используй непредвзятый и журналистский тон. Не повторяй текст.
Создай окончательный ответ ("FINAL ANSWER").
Не пытайся придумать ответ.
Отвечай только на русском языке за исключением специальных терминов.
Если документы не содержат ответа на вопрос, скажи, что "Я не могу ответить на вопрос на основе информации. Попробуйте переформулировать вопрос."
template_format: f-string
_type: prompt
```

#### Пользовательский промпт

```
input_variables: [question, summaries]
output_parser: null
template: "QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:"
template_format: f-string
_type: prompt
```

#### Промпт для обработки первоначального запроса пользователя

```
Вот запрос пользователя. Думай по шагам и оцени, что пользователь хотел узнать.
В качестве ответа выведи настоящий запрос, который хотел ввести пользователь.
```

#### Промпт для суммаризации документа

Если по запросу найдено много длинных чанков, можно попосить LLM суммаризовать каждый из нейденных чанков.

> ConversationSummaryMemory

#### Промпт для ранжирования результатов

Вместо использования реранкера можно попросить LLM отранжировать документы в контексте.

#### Промпт для реврайта

На выходе можно попросить LLM вывести сгенерированный результат в определенном формате.

## Метрики оценки RAG-системы

#### Формируем выборку для оценки

**Golden set** (вопросы и ответы) должен быть составлен при помощи естественного интеллекта. Лучше всего, если будет несколько экземпляров ответов на один вопрос.

#### Метрики поиска (ранжирования)

Не учитывающие порядок:
* Precision@K (P@K)

Учитывающие порядок:
* MAP@K
* NDCG@K

#### Метрики генерации

ROUGE, BERTScore, BLEURT, METEOR. Можно написать взвешенную сумму перечисленных метрик.

А еще можно попросить LLM вернуть не сгенерированные токены, а их логиты, и оценить, насколько она сама уверена в ответе (token level uncertainty).

## Проблемы в RAG-системах и способы их решения

* [12 RAG Pain Points and Solutions](https://originshq.com/blog/12-rag-pain-points-and-solutions/)
* [Seven Failure Points When Engineering a Retrieval Augmented Generation System](https://arxiv.org/pdf/2401.05856)